In [1]:
pip install opencv-python mediapipe numpy


Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import mediapipe as mp
import math

def calculate_angle(a, b, c):
    # Calculate the angle between three points using the dot product
    radians = math.atan2(c.y - b.y, c.x - b.x) - math.atan2(a.y - b.y, a.x - b.x)
    angle = abs(math.degrees(radians))
    return angle

def detect_pushups():
    cap = cv2.VideoCapture(0)
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    detection_threshold = 3  # Number of consecutive frames without detection before displaying "Unable to detect"
    detection_counter = 0

    # Display initial prompt
    print("Hello!! Are you ready to do pushups?")
    answer = input("Type 'yes' to continue: ")
    if answer.lower() != 'yes':
        cap.release()
        cv2.destroyAllWindows()
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks is not None:
            detection_counter = 0  # Reset detection counter
            left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]

            angle = calculate_angle(left_shoulder, right_shoulder, left_hip)

            if angle > 160:
                cv2.putText(frame, "Pushup", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            else:
                cv2.putText(frame, "Not a Pushup", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        else:
            detection_counter += 1
            if detection_counter >= detection_threshold:
                cv2.putText(frame, "Unable to detect", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow("Push-up Detection", frame)

        # Check for keyboard input
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('x'):
            print("Do you want to close the operation?")
            close_answer = input("Type 'yes' to close: ")
            if close_answer.lower() == 'yes':
                break

    cap.release()
    cv2.destroyAllWindows()

detect_pushups()


Hello!! Are you ready to do pushups?
Type 'yes' to continue: yes
Do you want to close the operation?
Type 'yes' to close: yes
